# MRI and CSF Mask Processing Playground

## Table of Contents

1. [Introduction](#Introduction)
2. [Setup](#Setup)
3. [Loading and Denoising MRI Data](#Loading-and-Denoising-MRI-Data)
4. [Intensity Normalization](#Intensity-Normalization)
5. [Feature Extraction](#Feature-Extraction)
6. [Clustering with K-Means](#Clustering-with-K-Means)
7. [Morphological Operations](#Morphological-Operations)
8. [Visualization](#Visualization)
9. [Saving Outputs](#Saving-Outputs)
10. [Conclusion](#Conclusion)


## Introduction

Welcome to the **MRI and CSF Mask Processing Playground**! This notebook provides an interactive environment to explore and experiment with the processing pipeline designed for MRI data and CSF mask segmentation. You can run individual cells to execute specific parts of the pipeline, visualize intermediate results, and tweak parameters as needed.

## Setup

### 1. Importing Necessary Libraries

First, ensure that all required libraries are installed. If not, you can install them directly from the notebook using `pip`.

In [ ]:
# Install required packages if not already installed
!pip install -r requirements.txt

### 2. Importing Modules

Import the necessary modules from your project. Ensure that your project's root directory is in the Python path so that the notebook can access `utils.py` and `config.py`.

In [ ]:
import sys
import os

# Add the project directory to the Python path
project_dir = os.path.abspath(os.path.join('..'))  # Adjust the path if necessary
if project_dir not in sys.path:
    sys.path.insert(0, project_dir)

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm

# Import utility functions and configurations
from utils import (
    load_image,
    denoise_image,
    normalize_intensity,
    compute_neighborhood_statistics,
    visualize_feature_distributions,
    analyze_feature_distributions,
    perform_kmeans,
    create_refined_masks,
    plot_refined_masks_on_slices,
    visualize_and_save_html,
    extract_and_decimate_meshes
)
from config import (
    ORIGINAL_MRI_PATH,
    DENOISED_TIFF_PATH,
    MASKS_DIR,
    OUTPUT_HTML,
    OUTPUT_SLICES_DIR,
    K,
    BATCH_SIZE,
    STRUCTURING_ELEMENT_RADIUS,
    MIN_SIZE
)

### 3. Setting Up Logging (Optional)

If you wish to enable logging within the notebook, set the logging parameters accordingly. This can help in monitoring the execution and memory usage.

In [ ]:
# Enable logging if desired
enable_logging = True
log_file = 'logs/execution.log'

if enable_logging:
    import logging
    setup_logging(enable_logging, log_file)

<hr>

## Loading and Denoising MRI Data

### 1. Loading the Original MRI Image

Load the original MRI image using the `load_image` function from `utils.py`.

In [ ]:
# Load the original MRI image
original_img = load_image(ORIGINAL_MRI_PATH)

### 2. Denoising the MRI Image

Denoise the loaded MRI image. If a denoised image already exists, it will be loaded; otherwise, the denoising process will be performed.

In [ ]:
# Denoise the MRI image
denoised_img = denoise_image(original_img, DENOISED_TIFF_PATH)

<hr>

## Intensity Normalization

Normalize the intensity of the denoised MRI image to ensure consistent analysis.

In [ ]:
# Normalize the intensity of the denoised image
denoised_img_normalized = normalize_intensity(denoised_img)

### 1. Visualizing a Normalized Slice

Plot a specific slice (e.g., slice index 90) to visualize the normalization effect.

In [ ]:
# Define the slice index to visualize
slice_index = 90  # Adjust based on your data

# Plot the normalized slice
if slice_index < denoised_img_normalized.shape[0]:
    plt.figure(figsize=(6, 6))
    plt.imshow(denoised_img_normalized[slice_index], cmap='gray')
    plt.title(f'Normalized Denoised MRI Slice {slice_index}')
    plt.axis('off')
    plt.show()
else:
    print(f"Slice index {slice_index} is out of bounds for image with {denoised_img_normalized.shape[0]} slices.")

<hr>

## Feature Extraction

### 1. Computing Neighborhood Statistics

Compute the neighborhood mean and variance for each voxel to enhance feature representation.

In [ ]:
# Compute neighborhood statistics
neighborhood_size = 3  # 3x3x3 neighborhood
neighborhood_mean, neighborhood_variance = compute_neighborhood_statistics(
    denoised_img_normalized,
    neighborhood_size=neighborhood_size
)

### 2. Preparing Features for Clustering

Flatten and combine the intensity, neighborhood mean, and variance into a feature matrix suitable for K-Means clustering.

In [ ]:
# Prepare features for K-Means
print("Preparing features for K-Means clustering...")
logging.info("Preparing features for K-Means clustering.")

intensity_flat = denoised_img_normalized.flatten().reshape(-1, 1)
mean_flat = neighborhood_mean.flatten().reshape(-1, 1)
variance_flat = neighborhood_variance.flatten().reshape(-1, 1)

features = np.hstack((intensity_flat, mean_flat, variance_flat)).astype(np.float32)
print(f"Features shape: {features.shape}")
logging.info(f"Features shape: {features.shape}")
print_memory_usage()
log_memory_usage()

### 3. Visualizing Feature Distributions

Plot the distributions of the extracted features to understand their characteristics.

In [ ]:
# Visualize feature distributions
visualize_feature_distributions(features)

### 4. Analyzing Feature Relationships

Use Seaborn's pairplot to analyze the relationships between different features.

In [ ]:
# Analyze feature distributions with pairplot
analyze_feature_distributions(features)

<hr>

## Clustering with K-Means

### 1. Performing K-Means Clustering

Apply MiniBatch K-Means to segment the MRI data into distinct clusters.

In [ ]:
# Perform K-Means clustering
kmeans = perform_kmeans(features, k=K, batch_size=BATCH_SIZE)

# Retrieve cluster labels
labels = kmeans.labels_

# Reshape labels back to the original image dimensions
clustered_img = labels.reshape(denoised_img_normalized.shape)
print("Cluster labels reshaped to image dimensions.")
logging.info("Cluster labels reshaped to image dimensions.")

<hr>

## Morphological Operations

### 1. Creating Refined Masks

Apply morphological operations to each cluster mask to remove small artifacts and fill holes.

In [ ]:
# Define the structuring element
from skimage.morphology import ball

selem = ball(STRUCTURING_ELEMENT_RADIUS)

# Create and refine masks for all clusters
refined_masks, csf_cluster = create_refined_masks(
    clustered_img,
    denoised_img_normalized,
    k=K,
    selem=selem,
    min_size=MIN_SIZE,
    masks_dir=MASKS_DIR
)

### 2. Visualizing Refined Masks on Slices

Overlay the refined masks onto a specific slice for visualization.

In [ ]:
# Define the slice index to visualize
slice_index = 90  # Adjust based on your data

# Plot refined masks overlaid on the specified slice
plot_refined_masks_on_slices(refined_masks, denoised_img_normalized, slice_index, K)

<hr>

## Visualization

### 1. Visualizing and Saving as HTML

Generate an interactive 3D visualization of the Original MRI, Denoised MRI, and refined CSF masks. Save the visualization as an HTML file for easy sharing and exploration.

In [ ]:
# Visualize and save as HTML
visualize_and_save_html(
    original_img,
    denoised_img_normalized,
    refined_masks,
    K,
    OUTPUT_HTML
)

<hr>

## Saving Outputs

### 1. Saving Specific Slices (Optional)

If you wish to save specific slices with overlayed masks, you can use the `plot_refined_masks_on_slices` function. This step was already performed in the **Morphological Operations** section. However, if you have additional slices to save, you can adjust the slice indices accordingly.

In [ ]:
# Example: Save an additional slice
additional_slice_index = 120  # Adjust based on your data
plot_refined_masks_on_slices(refined_masks, denoised_img_normalized, additional_slice_index, K)

<hr>